# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import vactionPy_apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "data/weather_df.csv"

weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
0,Vaini,TO,-21.2000,-175.2000,69.800,94,20,1.54
1,Tasiilaq,GL,65.6145,-37.6368,21.200,68,20,2.06
2,Itarema,BR,-2.9248,-39.9167,83.282,74,91,7.61
3,Hobart,AU,-42.8794,147.3294,50.774,71,8,4.12
4,Rikitea,PF,-23.1203,-134.9692,76.190,65,27,1.05
...,...,...,...,...,...,...,...,...
549,Asău,RO,46.4333,26.4000,39.110,48,40,0.51
550,Taksimo,RU,56.3444,114.8794,27.662,90,100,1.36
551,Gambo,CF,4.6482,22.2633,70.052,96,96,0.43
552,Sola,VU,-13.8833,167.5500,82.886,71,100,4.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=vactionPy_apiKey)

In [5]:
# Store 'Latitude' and 'Longitude' as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# 'Humidity' as float data type
humidity = weather_df["Humidity"].astype(float)

In [6]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
hotel_df = weather_df.loc[(weather_df['Temperature']>= 65)
                           & (weather_df['Temperature']< 80)
                            & (weather_df['Humidity']>= 10)
                           & (weather_df['Humidity']< 20)]
hotel_df.dropna(how='any')

hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
69,Awjilah,LY,29.1081,21.2869,73.814,18,0,4.39
128,Saint George,US,37.1041,-113.5841,74.516,13,1,6.18
439,Cañon City,US,38.4410,-105.2424,69.224,11,1,3.09
448,Guymon,US,36.6828,-101.4816,75.200,19,1,8.23
474,Lādwa,IN,29.9944,77.0444,73.202,13,0,2.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Google Places - hotel finder

hotel_list = []

for index, city in hotel_df.iterrows():
    #print(city[1], city[2])
    lat = city[2]
    lng = city[3]
    target_search = "lodging"
    params = {
    "location": f"{lat},{lng}",
    "types": target_search,
    "rankby": "distance",
    "key": vactionPy_apiKey
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    hotel_response = response.json()
    hotel_name = hotel_response["results"][0]["name"]
    hotel_list.append(hotel_name)

# create new column 'Hotel Name' and fill with generated hotels

hotel_df['Hotel Name'] = hotel_list

<ipython-input-8-60c1dc3921d1>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_list


In [9]:
# display Hotel data frame
hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH),Hotel Name
69,Awjilah,LY,29.1081,21.2869,73.814,18,0,4.39,استراحة التعاون أوجلة
128,Saint George,US,37.1041,-113.5841,74.516,13,1,6.18,Kelsch Metal Roofing
439,Cañon City,US,38.4410,-105.2424,69.224,11,1,3.09,Econo Lodge
448,Guymon,US,36.6828,-101.4816,75.200,19,1,8.23,Masonic Lodge
474,Lādwa,IN,29.9944,77.0444,73.202,13,0,2.70,House Of Rinkesh Garg


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
                                 info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))